<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lab-|-Tools-prompting" data-toc-modified-id="Lab-|-Tools-prompting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lab | Tools prompting</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#How-to-add-ad-hoc-tool-calling-capability-to-LLMs-and-Chat-Models" data-toc-modified-id="How-to-add-ad-hoc-tool-calling-capability-to-LLMs-and-Chat-Models-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>How to add ad-hoc tool calling capability to LLMs and Chat Models</a></span></li></ul></li><li><span><a href="#Setup" data-toc-modified-id="Setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Create-a-tool" data-toc-modified-id="Create-a-tool-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Create a tool</a></span></li><li><span><a href="#Creating-our-prompt" data-toc-modified-id="Creating-our-prompt-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Creating our prompt</a></span></li><li><span><a href="#Adding-an-output-parser" data-toc-modified-id="Adding-an-output-parser-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Adding an output parser</a></span></li><li><span><a href="#Invoking-the-tool-🏃" data-toc-modified-id="Invoking-the-tool-🏃-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Invoking the tool 🏃</a></span></li><li><span><a href="#Let's-put-it-together" data-toc-modified-id="Let's-put-it-together-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Let's put it together</a></span></li><li><span><a href="#Returning-tool-inputs" data-toc-modified-id="Returning-tool-inputs-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Returning tool inputs</a></span></li><li><span><a href="#What's-next?" data-toc-modified-id="What's-next?-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>What's next?</a></span></li></ul></li></ul></div>

# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [32]:
%pip install --upgrade --quiet langchain langchain-community

In [33]:
!pip install ollama openai

If you'd like to use LangSmith, uncomment the below:

In [34]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [35]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [36]:
!rm myscript.sh
!echo '#!/bin/bash' > myscript.sh
!echo 'ollama serve' >> myscript.sh
# Make the script executable
!chmod +x myscript.sh
!nohup ./myscript.sh &

nohup: appending output to 'nohup.out'


In [37]:
#running model:
!nohup ollama run phi3:medium > ollama_model.log 2>&1 &

In [38]:
#pulling embeddings:
!nohup ollama pull phi3:medium > ollama_emd_model.log 2>&1 &

In [39]:
!ollama list

NAME       	ID          	SIZE  	MODIFIED       
phi3:mini  	d184c916657e	2.2 GB	17 minutes ago	
phi3:medium	1e67dff39209	7.9 GB	17 minutes ago	


You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](/docs/integrations/chat/), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](/docs/integrations/chat/ollama/).

In [40]:
# from langchain_community.llms import Ollama

# model = Ollama(model="phi3")



## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](/docs/how_to/custom_tools).

In [41]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [42]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [99]:
import ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import Runnable
from langchain_core.tools import render_text_description
import json

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [103]:
class OllamaLLM(Runnable):
    def __init__(self, model_name):
        self.model_name = model_name

    def invoke(self, input, config=None):
        # Handle ChatPromptValue object
        if hasattr(input, "to_string"):
            user_input = input.to_string()
        else:
            user_input = input.get("input", "")

        # Define the system message content
        system_content = f"""
          You are an assistant that has access to the following set of tools.
          Here are the names and descriptions for each tool:
          {rendered_tools}
          Given the user input, return the name and input of the tool to use.
          Return your response as a JSON blob with 'name' and 'arguments' keys.
          The 'arguments' should be a dictionary, with keys corresponding
          to the argument names and the values corresponding to the requested values.
          Ensure the arguments are named correctly according to the tool's requirements:
          - For the add tool, arguments must be named: 'x' and 'y'
        """

        response = ollama.chat(
            model=self.model_name,
            messages=[
                {
                    'role': 'system',
                    'content': system_content
                },
                {
                    'role': 'user',
                    'content': user_input
                }
            ]
        )

        # EDIT: Ensure the response content is a string and properly formatted
        return response['message']['content']

# Example usage
model = OllamaLLM(model_name="phi3:medium")
response = model.invoke({"input": "what's 3 plus 1132"})
print(response)  # Print the response directly for debugging


 {
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 1132
  }
}


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [94]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Create the prompt template
prompt = ChatPromptTemplate.from_template(f"""
[system]
{system_content}

[user]
{{input}}
""")

# Set up the chain with the prompt, model, and JsonOutputParser
chain = prompt | model | JsonOutputParser()

# Invoke the chain with the input
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'x': 13, 'n': 4}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [95]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [96]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [97]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [104]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).